In [1]:
import torch
import torch.nn as nn

In [4]:
class InputBlock(nn.Module):
    def __init__(self, in_channels = None, out_channels = None):
        super(InputBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
    
    def forward(self, x):
        pass

In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels = None, out_channels = None):
        super(ResidualBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        
    def forward(self, x):
        pass

In [6]:
class MiddleBlock(nn.Module):
    def __init__(self, in_channels = None, out_channels = None):
        super(MiddleBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        
    def forward(self, x, skip_info):
        pass

In [7]:
class UpSampleBlock(nn.Module):
    def __init__(self, in_channels = None, out_channels = None):
        super(UpSampleBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        
    def forward(self, x):
        pass

In [8]:
class OutputBlock(nn.Module):
    def __init__(self, in_channels = None, out_channels = None):
        super(OutputBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        
    def forward(self, x):
        pass

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
    def forward(self, x):
        pass